In [1]:
%matplotlib inline
import pandas as pd #pandas is a useful array/matrix/spreadsheet maipulator. But it has some unique language
import numpy as np #math stuff!
import matplotlib.pyplot as plt #plotting package.
import seaborn as sns
from mpl_toolkits.basemap import Basemap
from sklearn.ensemble import RandomForestClassifier
from scipy import sparse
import scipy

In [ ]:
#This Notebook takes the raw train and test sets and converts them into numbers.
#Along the way, it bins the longitude and latitude values into a grid,
#separates the address section into 2 streets and numericalizes the most common,
#separates the Dates into date and time values, and
#chucks data in the training set that's not in the test set

#To do make sure everything I do to the test set I do to the training set,
#I first combine the 2 sets together so operations act on both at the same time.

In [2]:
#We import both the training and test data
Train=pd.read_csv('train.csv', header=0)
Test=pd.read_csv('test.csv', header=0)

In [3]:
Train.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [3]:
#Now, I drop some features that are in the training data but aren't in the test data
Train=Train.drop('Descript', axis=1)
Train=Train.drop('Resolution',axis=1)
#I also insert a column that is only in the test data:
Train.insert(0,'Id',-1,allow_duplicates=False)
Train_Length=len(Train) #This may be useful for breaking the Train and Test data apart.

In [77]:
Train.head(2)

,Id,Dates,Category,DayOfWeek,PdDistrict,Address,X,Y
0,-1,2015-05-13 23:53:00,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599
1,-1,2015-05-13 23:53:00,OTHER OFFENSES,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599


In [4]:
#Now, I add a 'result' column into the Test data. I will delete before finishing,
#but it will help to have a dummy variable in there right now.
Test.insert(2,'Category','Fake',allow_duplicates=False)
Test_Length=len(Test)

In [79]:
Test.head(2)

,Id,Dates,Category,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Fake,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Fake,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432


In [5]:
#At this point, I can combine the Train and Test data
#Now any new categories being made will be made on both
Full_Data=pd.concat([Train,Test])
#Let's reset the index to make operations easier.
Full_Data=Full_Data.reset_index(drop=True)

In [81]:
Full_Data.head(2)

,Id,Dates,Category,DayOfWeek,PdDistrict,Address,X,Y
0,-1,2015-05-13 23:53:00,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599
1,-1,2015-05-13 23:53:00,OTHER OFFENSES,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599


In [82]:
Full_Data.tail(2)

,Id,Dates,Category,DayOfWeek,PdDistrict,Address,X,Y
1762309,884260,2003-01-01 00:01:00,Fake,Wednesday,BAYVIEW,1500 Block of HUDSON AV,-122.387394,37.739479
1762310,884261,2003-01-01 00:01:00,Fake,Wednesday,TARAVAL,1500 Block of SLOAT BL,-122.489714,37.733950


In [6]:
#Now, we can start manipulating the data.
#First, binning X and Y:
#Using measurements from png map of San Fran, I make equal sized bins for lon and lat:
lon_lat_box = (-122.5226, -122.3496, 37.7007, 37.8152);
binsLon = np.linspace(lon_lat_box[0], lon_lat_box[1], 10);
binsLat = np.linspace(lon_lat_box[2], lon_lat_box[3], 10);

#I'm sure there's a quicker way to make these, but this works.
Lon_boxes=[];
for x in range(1,len(binsLon)):
    Lon_boxes.append(x)
Lat_boxes=[];
for x in range(1,len(binsLat)):
    Lat_boxes.append(x)
#First, I divide up all entries based on longitude values:
Full_Data['X_binned']=pd.cut(Full_Data['X'], binsLon, labels=Lon_boxes)
Full_Data['Y_binned']=0 #So values outside my box described by binsLat and binsLon will be mapped here
#Now for each longitude bin, I break up into lat bins:
for x in range(1,len(Lon_boxes)):
    a1=pd.cut(Full_Data[Full_Data['X_binned']==x]['Y'], binsLat, labels=Lat_boxes)
    Full_Data.loc[a1.index.values,'Y_binned']=a1.values

In [7]:
#There are some values which didn't get placed in the bins, so let's set those as the most common bin values:
Full_Data['Y_binned'].fillna(6,inplace = True);
Full_Data['X_binned'].fillna(6,inplace = True);

In [85]:
Full_Data.head(1)

,Id,Dates,Category,DayOfWeek,PdDistrict,Address,X,Y,X_binned,Y_binned
0,-1,2015-05-13 23:53:00,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,6,6.0


In [8]:
#Looking at addresses, they are either intersections or the block of a road.
#I choose to break intersections into each street and seperate blocks and their road.
#First, replacing all the 'of' with '/' (e.g. 600 block of Market ST-->600 block / Market ST)
Full_Data['Address1']=Full_Data['Address'].str.replace('of','/')
#Now make 2 new columns using split on the ' / ':
#(Spaces added above to remove empty spaces from new entries)
Full_Data[['Address2','Address3']]=Full_Data['Address1'].str.split(' / ',expand=True)

In [9]:
#Doing the same on the dates:
Full_Data[['Date','Time']]=Full_Data['Dates'].str.split(' ', expand=True)
Full_Data[['Year','Month','Day']]=Full_Data['Date'].str.split('-', expand=True)
Full_Data[['Hour','Minute','Second']]=Full_Data['Time'].str.split(':',expand=True)

In [10]:
#These are technically still strings, which won't be good for machine learning.
#Converting to numbers:
Full_Data['Year']=pd.to_numeric(Full_Data['Year']);
Full_Data['Month']=pd.to_numeric(Full_Data['Month']);
Full_Data['Day']=pd.to_numeric(Full_Data['Day']);
Full_Data['Hour']=pd.to_numeric(Full_Data['Hour']);
Full_Data['Minute']=pd.to_numeric(Full_Data['Minute']);

In [90]:
Full_Data.head(1)

,Id,Dates,Category,DayOfWeek,PdDistrict,Address,X,Y,X_binned,Y_binned,...,Address2,Address3,Date,Time,Year,Month,Day,Hour,Minute,Second
0,-1,2015-05-13 23:53:00,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,6,6.0,...,OAK ST,LAGUNA ST,2015-05-13,23:53:00,2015,5,13,23,53,00


In [12]:
#We Have far too many street names as well, and we need to make them into numbers:
a1=pd.value_counts(Full_Data['Address2'])[0:21] #Picking only the most common street names.
#Here, I've set cutoff to be 10% of most common name.
a2=a1.index.values #picking the values
a3=pd.Categorical(Full_Data['Address2'], categories=a2, ordered=False) #Finding only those streets in the dataset
Full_Data['Address2_1']=a3 #making new column with only those streets and NaN for all others
Full_Data['Address2_2']=100; #new column with starting value that will represent NaN from previous column
a1=pd.value_counts(Full_Data['Address2_1']); #Getting the non NaN entries
a2=a1.index.values; 
#below, giving each street a numerical value
for x in range(0,len(a2)):
    Full_Data.loc[Full_Data[Full_Data['Address2_1']==a2[x]].index.values,'Address2_2']=x

a1=pd.value_counts(Full_Data['Address3'])[0:44] #Picking only the most common street names.
#Here, I've set cutoff to be 10% of most common name.
a2=a1.index.values #picking the values
a3=pd.Categorical(Full_Data['Address3'], categories=a2, ordered=False) #Finding only those streets in the dataset
Full_Data['Address3_1']=a3 #making new column with only those streets and NaN for all others
Full_Data['Address3_2']=100; #new column with starting value that will represent NaN from previous column
a1=pd.value_counts(Full_Data['Address3_1']); #Getting the non NaN entries
a2=a1.index.values; 
#below, giving each street a numerical value
for x in range(0,len(a2)):
    Full_Data.loc[Full_Data[Full_Data['Address3_1']==a2[x]].index.values,'Address3_2']=x

In [13]:
#And making day of the week into a number
Full_Data['Weekday']=8;
a1=pd.value_counts(Full_Data['DayOfWeek']);
a2=a1.index.values;
for x in range(0,len(a2)):
    Full_Data.loc[Full_Data[Full_Data['DayOfWeek']==a2[x]].index.values,'Weekday']=x

In [14]:
#Finally, let's do the same with the category section
Full_Data['Outcome']=100;
a1=pd.value_counts(Train['Category']);
a2=a1.index.values;
for x in range(0,len(a2)):
    Full_Data.loc[Full_Data[Full_Data['Category']==a2[x]].index.values,'Outcome']=x

In [19]:
#We need to record what this mapping looked like, so...what does it look like?
Full_Data[['Category','Outcome']][0:10]

,Category,Outcome
0,WARRANTS,7
1,OTHER OFFENSES,1
2,OTHER OFFENSES,1
3,LARCENY/THEFT,0
4,LARCENY/THEFT,0
5,LARCENY/THEFT,0
6,VEHICLE THEFT,5
7,VEHICLE THEFT,5
8,LARCENY/THEFT,0
9,LARCENY/THEFT,0


In [20]:
#oh look, it's just in the order of a2. Excellent!
a2

array(['LARCENY/THEFT', 'OTHER OFFENSES', 'NON-CRIMINAL', 'ASSAULT',
       'DRUG/NARCOTIC', 'VEHICLE THEFT', 'VANDALISM', 'WARRANTS',
       'BURGLARY', 'SUSPICIOUS OCC', 'MISSING PERSON', 'ROBBERY', 'FRAUD',
       'FORGERY/COUNTERFEITING', 'SECONDARY CODES', 'WEAPON LAWS',
       'PROSTITUTION', 'TRESPASS', 'STOLEN PROPERTY',
       'SEX OFFENSES FORCIBLE', 'DISORDERLY CONDUCT', 'DRUNKENNESS',
       'RECOVERED VEHICLE', 'KIDNAPPING', 'DRIVING UNDER THE INFLUENCE',
       'RUNAWAY', 'LIQUOR LAWS', 'ARSON', 'LOITERING', 'EMBEZZLEMENT',
       'SUICIDE', 'FAMILY OFFENSES', 'BAD CHECKS', 'BRIBERY', 'EXTORTION',
       'SEX OFFENSES NON FORCIBLE', 'GAMBLING', 'PORNOGRAPHY/OBSCENE MAT',
       'TREA'], dtype=object)

In [21]:
#Let's drop the features we no longer need:
Full_Data.columns.values

array(['Id', 'Dates', 'Category', 'DayOfWeek', 'PdDistrict', 'Address',
       'X', 'Y', 'X_binned', 'Y_binned', 'Address1', 'Address2',
       'Address3', 'Date', 'Time', 'Year', 'Month', 'Day', 'Hour',
       'Minute', 'Second', 'Address2_1', 'Address2_2', 'Address3_1',
       'Address3_2', 'Weekday', 'Outcome'], dtype=object)

In [22]:
#For now, I just choose the numerical numbers I am interested in.
Full_Data=Full_Data[['Id','Outcome','Weekday','Address3_2','Address2_2','Year','Month','Day','Hour','Minute','X_binned','Y_binned']]

In [23]:
Full_Data.head(2)

,Id,Outcome,Weekday,Address3_2,Address2_2,Year,Month,Day,Hour,Minute,X_binned,Y_binned
0,-1,7,1,100,100,2015,5,13,23,53,6,6.0
1,-1,1,1,100,100,2015,5,13,23,53,6,6.0


In [24]:
#Now we need to split everything up again
TrainNum=Full_Data[Full_Data['Id']<0]
TestNum=Full_Data[Full_Data['Id']>=0]

In [25]:
len(TrainNum)==Train_Length

True

In [26]:
len(TestNum)==Test_Length

True

In [105]:
TrainNum.head(2)

,Id,Outcome,Weekday,Address3_2,Address2_2,Year,Month,Day,Hour,Minute,X_binned,Y_binned
0,-1,7,1,100,100,2015,5,13,23,53,6,6.0
1,-1,1,1,100,100,2015,5,13,23,53,6,6.0


In [27]:
TrainNum=TrainNum.drop('Id', axis=1)

In [107]:
TestNum.head(2)

,Id,Outcome,Weekday,Address3_2,Address2_2,Year,Month,Day,Hour,Minute,X_binned,Y_binned
878049,0,100,6,100,16,2015,5,10,23,59,7,3.0
878050,1,100,6,100,100,2015,5,10,23,51,7,3.0


In [28]:
TestNum=TestNum.drop('Outcome',axis=1)
TestNum=TestNum.reset_index(drop=True)

In [29]:
#OK, let's write these new dataframes into csv's so I can save them.
TrainNum.to_csv('TrainNum.csv',header=True, index=False)
TestNum.to_csv('TestNum.csv',header=True, index=False)